# **Procesamiento de Lenguaje Natural**

## Maestría en Inteligencia Artificial Aplicada
#### Tecnológico de Monterrey
#### Prof Luis Eduardo Falcón Morales

### **Actividad en Equipos: sistema LLM + RAG**

* **Nombres y matrículas:**

  *   Arturo Jain Delgadillo A01794992
  *   Ramiro Martin Jaramillo Romero A01171251
  * José Ashamat Jaimes Saavedra A01736690
  *   Owen Jáuregui Borbón A01638122

* **Número de Equipo:**
  *   Equipo 80


* ##### **El formato de este cuaderno de Jupyter es libre, pero debe incuir al menos las siguientes secciones:**

  * ##### **Introducción de la problemática a resolver.**
  * ##### **Sistema RAG + LLM**
  * ##### **El chatbot, incluyendo ejemplos de prueba.**
  * ##### **Conclusiones**

* ##### **Pueden importar los paquetes o librerías que requieran.**

* ##### **Pueden incluir las celdas y líneas de código que deseen.**

# Introducción de la problemática a resolver.

El personal de salud en México debe cumplir con muchas Normas Oficiales Mexicanas (NOM). Estas normas cubren higiene, bioseguridad, infraestructura y calidad. Incumplirlas puede traer multas, clausuras e incluso poner en riesgo a los pacientes.

Hay tres problemas frecuentes:

* Encontrar la norma correcta entre docenas de documentos y versiones.

* Entender el lenguaje jurídico y técnico y convertirlo en pasos claros para el día a día.

* Seguir las actualizaciones que publica el Diario Oficial.

Clínicas pequeñas, laboratorios y comités internos de calidad suelen trabajar sin apoyo legal especializado. Esto provoca retrasos en auditorías y riesgos de sanción.

Para cerrar esa brecha proponemos un chatbot que combina un modelo de lenguaje grande con RAG. El sistema carga 21 NOM de salud (por ejemplo la NOM-007 sobre atención obstétrica y la NOM-059 sobre buenas prácticas de fabricación). Cuando alguien hace una pregunta en lenguaje natural, el chatbot busca los párrafos más cercanos y genera una respuesta en español con la cita original.

Así el personal puede:

* Consultar requisitos concretos.

* Recibir explicaciones sencillas con la referencia al texto oficial.

* Acceder a la versión vigente sin buscar archivos PDF.

Con esto mejoramos el acceso a la normativa, reducimos errores de cumplimiento y apoyamos la capacitación continua en los servicios de salud.

# Sistema RAG + LLM

In [ ]:
# Incluyan a continuación todas las celdas (de código o texto) que deseen...

!pip uninstall -y transformers
!pip -q install --upgrade \
  langchain langchain-community langchain-huggingface \
  faiss-cpu sentence-transformers \
  bitsandbytes accelerate \
  pypdf "gradio>=4.17.0"
!pip install --no-cache-dir git+https://github.com/huggingface/transformers.git

Found existing installation: transformers 4.53.0.dev0
Uninstalling transformers-4.53.0.dev0:
  Successfully uninstalled transformers-4.53.0.dev0
  Cloning https://github.com/huggingface/transformers.git to /tmp/pip-req-build-5bcylz79
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers.git /tmp/pip-req-build-5bcylz79
  Resolved https://github.com/huggingface/transformers.git to commit 2166b6b4ff09f6dd3867ab982f262f66482aa968
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for transformers: filename=transformers-4.53.0.dev0-py3-none-any.whl size=11461635 sha256=d04cab7673c5f582d544a63ef9e5fce4649cb3838adf3ea5063dd8935d3c9d51
  Stored in directory: /tmp/pip-ephem-wheel-cache-e8h2jcgl/wheels/32/4b/78/f195c684dd3a9ed21f3b39fe8f85b48df7918581b6437be143
Successfully built transformers
  Attempting uninstall: transformers
    Found existing insta

In [ ]:
from google.colab import drive
import os, warnings, gc, torch

drive.mount('/content/drive')
DATA_DIR = '/content/drive/MyDrive/2025-1/NOM_PDFs'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# ---------------------------------------------------------------
# 1. Cargar y fragmentar los PDFs
# ---------------------------------------------------------------

# LangChain es un framework de trabajo que se esta posicionando como lider en la creación de arquitecturas/agentes
# RAG, RAFT en forma de cadena (LangChain) y/o agentes, agentes multitooling, redes de agentes, multiagentes en
# forma de grafo (LangGraph). Estos agentes en grafo, tienen la capacidad de elegir, con base en el query
# sus acciones, tareas y/o pasos a seguir.
# LangChain/LangGraph, nos brinda todas las herramientas para lograr estos procesos, e incluso
# cuenta con LangSmith, que permite observar lo que pasa dentro del modelo (cuando esta el procesamiento)

# RecursiveCharacterTextSplitter: sirve para dividir textos largos en partes más pequeñas.
# Utilizamos el framework Langchain para cargar los documentos PDF y para separar los documentos finales.
from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

# Obtenemos la lista de archivos en la carpeta DATA_DIR que terminan en ".pdf"
pdf_files = [f for f in os.listdir(DATA_DIR) if f.lower().endswith('.pdf')]

# Si no se encontró ningún archivo PDF, mostramos un error.
if not pdf_files:
    raise ValueError(f"No se encontraron PDFs en {DATA_DIR}.")

# Creamos una lista de "lectores de PDF", uno por cada archivo encontrado.
loaders = [PyPDFLoader(os.path.join(DATA_DIR, f)) for f in pdf_files]

# Leemos todos los archivos PDF y los convertimos en una lista de documentos.
# Cada documento representa una página del PDF.
docs = sum([ld.load() for ld in loaders], [])
print(f"Se leyeron {len(docs)} páginas de {len(loaders)} PDFs")

# Preparamos una herramienta para dividir los textos largos en fragmentos más pequeños.
# Cada fragmento tendrá hasta 768 caracteres, y los fragmentos se superponen 128 caracteres.
splitter = RecursiveCharacterTextSplitter(chunk_size=768, chunk_overlap=128)

# Aplicamos la división a todos los documentos cargados.
docs_split = splitter.split_documents(docs)
print(f"Total de fragmentos: {len(docs_split)}")

Se leyeron 323 páginas de 21 PDFs
Total de fragmentos: 1625


In [ ]:
# ---------------------------------------------------------------
# 2. Embeddings + índice FAISS
# ---------------------------------------------------------------

# ---------------------------------------------------------------
# 2. Crear representaciones numéricas (embeddings) y construir un índice de búsqueda (FAISS)
# ---------------------------------------------------------------

# Importamos FAISS, que permite crear un índice para buscar información rápidamente.
# En general FAISS funcionará como una BBDD de vectores, otras opciones pueden ser ChromaDB
# Estas BBDD vectoriales (para el almacenamiento de embeddings), permiten busqueda por similitud y busquedas avanzadas.
# Importamos SentenceTransformer, una herramienta para convertir texto en números.
# Importamos la clase base de Langchain para crear "embeddings personalizados".
from langchain_community.vectorstores import FAISS
from sentence_transformers import SentenceTransformer
from langchain.embeddings.base import Embeddings

# Definimos el nombre del modelo que se va a usar para transformar texto en números.
# Este modelo se llama 'all-MiniLM-L6-v2', es ligero pero bastante preciso.
EMB_MODEL = 'sentence-transformers/all-MiniLM-L6-v2'


emb_device = 'cpu'
print('⚡️  Cargando embeddings **E5-base** en CPU (más rápidos)…')

# Cargamos el modelo de embeddings en la CPU.
emb_model = SentenceTransformer(EMB_MODEL, device=emb_device)

# Creamos una clase personalizada llamada E5Embeddings.
# Esta clase se usa para transformar textos en "embeddings" (números que representan el significado del texto).
class E5Embeddings(Embeddings):
    def embed_documents(self, texts):
        # Convierte una lista de textos en listas de números.
        return emb_model.encode(
            texts,
            batch_size=32,                  # Procesa 32 textos a la vez
            show_progress_bar=True,         # Muestra una barra de progreso
            normalize_embeddings=True       # Normaliza los vectores para compararlos
        ).tolist()

    def embed_query(self, text):
        # Convierte una sola pregunta o frase en una lista de números.
        return emb_model.encode(
            text,
            show_progress_bar=False,        # No muestra progreso porque es solo una frase
            normalize_embeddings=True
        ).tolist()


embeddings = E5Embeddings()

# Creamos un índice de búsqueda usando FAISS.
# Este índice permite hacer búsquedas rápidas dentro de los fragmentos de texto previamente creados.
faiss_index = FAISS.from_documents(docs_split, embeddings)

print('✅ Índice FAISS listo')


⚡️  Cargando embeddings **E5-base** en CPU (más rápidos)…


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Batches:   0%|          | 0/51 [00:00<?, ?it/s]

✅ Índice FAISS listo


In [ ]:
# ---------------------------------------------------------------
# 3. Cargar LLM
# ---------------------------------------------------------------

# Importamos herramientas de Hugging Face:
# - AutoTokenizer: convierte texto en un formato que el modelo puede entender.
# - AutoModelForSeq2SeqLM: es el modelo de lenguaje que puede generar o transformar texto.
# También importamos HuggingFacePipeline, que permite usar estos modelos dentro del flujo de Langchain.
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, pipeline
from langchain_huggingface import HuggingFacePipeline

# Elegimos un modelo llamado "google/flan-t5-base".
# Este modelo es relativamente pequeño (250 MB) y funciona bien en una computadora sin tarjeta gráfica (CPU).
LLM_ID = "google/flan-t5-base"

# Cargamos el "tokenizer", que es como un traductor que prepara el texto para que el modelo lo entienda.
tokenizer = AutoTokenizer.from_pretrained(LLM_ID)

# Cargamos el modelo de lenguaje ya entrenado, listo para usar.
model = AutoModelForSeq2SeqLM.from_pretrained(LLM_ID)  # Se mantendrá funcionando en CPU

# Creamos un "pipeline", que es una forma fácil de usar el modelo para generar texto.
text_gen = pipeline(
    "text2text-generation",     # Indicamos que vamos a generar texto a partir de texto
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=128,         # El texto generado tendrá como máximo 128 palabras nuevas
    do_sample=True,             # Activamos la aleatoriedad en las respuestas (más variado)
    temperature=0.7,            # Controla qué tan creativas son las respuestas (0.7 es un balance)
    top_p=0.9                   # Limita las palabras posibles a las más probables (para mantener coherencia)
)


llm = HuggingFacePipeline(pipeline=text_gen)
print("✅ LLM cargado")



Device set to use cpu


✅ LLM cargado


In [ ]:
# ---------------------------------------------------------------
# 4. RAG con LangChain
# ---------------------------------------------------------------

# La arquitectura RAG (generación aumentada por recuperación), nos permite optimizar la
# salida de un Modelo de lenguaje, de forma que su respuesta, haga referencia a una
# base de concimientos especifica. Prácticamente, se encarga de extender las capacidades
# de un LLM de responder sobre información especifca.

# Este proceso se puede ver de la siguiente forma:
# 1.- Recibimos el query del usuario
# 2.- Recuperamos información relevante del query del usuario (retriever) (información previamente dividida y sus embbedings)
# 3.- Aumentamos el query del usuario con la información relevante y enviamos el query (prompt) al LLM
# 4.- Regresamos al usuario la respuesta del LLM.

# Importamos herramientas necesarias de LangChain:
# - RetrievalQA: permite hacer preguntas y obtener respuestas usando tanto un modelo de lenguaje como un buscador de información.
# - StreamingStdOutCallbackHandler: sirve para imprimir las respuestas poco a poco mientras se generan.
from langchain.chains import RetrievalQA
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler

# Creamos un "streamer" que mostrará el texto en pantalla mientras se genera, en tiempo real.
streamer = StreamingStdOutCallbackHandler()

# Creamos un "retriever" (buscador) a partir del índice FAISS creado antes.
# Este buscará los fragmentos más relevantes para responder una pregunta.
# "k=1" significa que solo se recuperará **un fragmento de texto** por pregunta.
# Esto se hace porque el modelo que estamos usando no puede manejar textos muy largos (solo 512 tokens).
retriever = faiss_index.as_retriever(search_kwargs={"k": 1}) # muy importante, k=1 ya que el modelo utilizado tiene límite de 512 tokens, esto produce resultados "meh", con mejor modelo, mejores resultados, se elige este modelo debido a que queremos probar todo "local"

# Creamos una cadena QA que combina:
# - El modelo de lenguaje que ya cargamos (llm)
# - El "retriever" que busca el contenido más relevante
# - Una estrategia llamada "stuff" que simplemente pasa el texto encontrado tal cual al modelo
# - La opción de devolver también el texto fuente (útil para saber de dónde salió la respuesta)
# - El "streamer" para mostrar la respuesta mientras se genera
# - verbose=True para ver información detallada del proceso
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type='stuff',
    retriever=retriever,
    return_source_documents=True,
    callbacks=[streamer],
    verbose=True
)
print('Cadena RAG inicializada')

Cadena RAG inicializada


# Chatbot, incluyendo ejemplos de prueba

In [ ]:
import gradio as gr

print(llm("Hola", max_new_tokens=8))

# Definimos una función que manejará la interacción del usuario con el chatbot.
# Esta función recibe una pregunta, busca la respuesta con RAG y muestra la fuente.
def rag_chat(text):
    print(text)  # Imprime la pregunta en la consola.

    # Usa la cadena RAG (búsqueda + generación) para obtener una respuesta basada en los PDFs
    out = qa_chain({'query': text})
    print(out)

    answer = out['result']
    print(answer)

    # Extrae los nombres de los archivos PDF que sirvieron como fuente de la respuesta.
    # Elimina duplicados y los ordena alfabéticamente.
    srcs = '\n'.join(sorted({
        os.path.basename(d.metadata['source'])
        for d in out['source_documents']
    }))
    return f"{answer}\n\n📄 **Fuentes consultadas:**\n{srcs}"

# Creamos una interfaz gráfica con Gradio
demo = gr.Interface(
    fn=rag_chat,  # Función que se ejecuta cuando el usuario hace una pregunta
    inputs=gr.Textbox(lines=2, label='Pregunta', value="¿Quién publica las NOM?"),  # Entrada de texto con una pregunta inicial
    outputs=gr.Markdown(label='Respuesta'),  # Salida en formato de texto con estilo Markdown
    title='Chatbot RAG sobre Normas Oficiales Mexicanas',
    description='Ingresa una pregunta y obtén una respuesta fundamentada en los PDFs de las NOM.'
)

demo.queue()
demo.launch(share=True, debug=True)

Hola, Hola, Hola, Hola, Hola, Hola, Hola, Hola, Hola, Hola, Hola, Hola, Hola, Hola, Hola, Hola, Hola, Hola, Hola, Hola, Hola, Hola, Hola, Hola, Hola, Hola, Hola, Hola, Hola, Hola, Hola, Hola, Hola, Hola, Hola, Hola, Hola, Hola, Hola, Hola, Hola, Hola, Hola
Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://1e37ea4cc4fd805396.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


¿Quién publica las NOM?


> Entering new RetrievalQA chain...

> Finished chain.
{'query': '¿Quién publica las NOM?', 'result': 'El Comité Consultivo Nacional de Normalizaci ón de Innovación, Desarrollo, Tecnologas e Información en Salud', 'source_documents': [Document(id='34554267-3103-4aba-8e6a-13a59a41e9b4', metadata={'producer': 'Microsoft® Word 2013', 'creator': 'Microsoft® Word 2013', 'creationdate': '2017-02-21T08:44:09-06:00', 'title': '', 'author': 'DOF', 'moddate': '2017-02-21T08:44:09-06:00', 'source': '/content/drive/MyDrive/2025-1/NOM_PDFs/037ssa32017.pdf', 'total_pages': 13, 'page': 0, 'page_label': '1'}, page_content='Que durante el periodo de cons ulta pública, que concluyó el 1o.  de noviembre del  2014, fueron recibidos \nen la sede del citado Comité, los comentarios formulados por los interesados respecto del proyecto de la \nNorma Oficial Mexicana, razón por la cual, con fecha previa fueron publicadas en el Diario Oficial de  la \nFederación las respue stas a los mi

# **Conclusiones:**

**El RAG marca la diferencia**

Usar recuperación de fragmentos resultó clave. El chatbot no “inventa” requisitos. Muestra el párrafo exacto de la NOM y luego lo explica en lenguaje claro. Esto genera confianza en el usuario. El uso de RAG permite que el modelo de información actualizada y relevante al contexto específico.

La posibilidad de aumentar los datos con cualquier archivo hace que el modelo sea pertinente al responder a una mayor variedad de prompts. Además de que el modelo sea portable a más casos específicos al no estar ligado a esta información externa.

**Un solo punto de consulta ahorra tiempo**

Antes el personal debía abrir varios PDF y buscar a mano. Ahora obtiene la respuesta en menos de un minuto. Esto mejora la eficiencia durante auditorías internas y capacitaciones.

**Mdelos más grandes dan mejores respuestas**

Con un modelo pequeño las respuestas son limitadas. La calidad del LLM influye mucho cuando la pregunta es compleja.


**Límites actuales**

* No detecta si la NOM fue derogada o modificada después de la fecha de corte.

* No responde consultas que mezclan normativa con procedimientos clínicos que no están en las NOM.


En conjunto, la actividad mostró que un chatbot LLM + RAG es una solución viable para acercar la normativa de salud al personal operativo. Se logró reducir la carga de búsqueda, aumentar la claridad de la información y sentar las bases para mejoras futuras.

Este prototipo demuestra un camino claro hacia herramientas más robustas, capaces de integrarse con fuentes normativas actualizadas y expandirse a nuevas áreas temáticas. Con mejoras progresivas en el modelo y la cobertura documental, esta solución puede convertirse en un eje estratégico para fortalecer el cumplimiento, la formación continua y la toma de decisiones basadas en evidencia normativa.

# **Fin de la actividad chatbot: LLM + RAG**